In [27]:
import numpy as np
import pandas as pd
import pathlib
from sklearn.preprocessing import OneHotEncoder

import models.pipelineLibrary
import models.utilsCode

In [2]:
#Parameters
seed = 1

train_validation_perc = 0.7
window = 4


input_filename = '../../dados_anon_fulldataset.csv'
input_filename = "../../dados10000.csv"

In [3]:
models.utilsCode.set_seed(seed)

In [4]:
df = pd.read_csv(input_filename)

In [5]:
df = df.drop(columns=['Unnamed: 0'])\
        .drop_duplicates()\
        .copy()

In [6]:
new_state = True
alunos_list = df['ID_CURSO_ALUNO'].unique()
if (new_state):
    alunos_train_validation = np.random.choice(len(alunos_list), size=int(len(alunos_list)*train_validation_perc), replace=False)
    np.save('random_state_.npy', alunos_train_validation)
else:
    alunos_train_validation = np.load('random_state_.npy')
data_train_validation_ = df[df['ID_CURSO_ALUNO'].isin(alunos_list[alunos_train_validation])]
data_test_ = df[~df['ID_CURSO_ALUNO'].isin(alunos_list[alunos_train_validation])]

alunos_list_train_validation = data_train_validation_['ID_CURSO_ALUNO'].unique()
alunos_list_test = data_test_['ID_CURSO_ALUNO'].unique()

# Train data
data_train = df[df['ID_CURSO_ALUNO'].isin(alunos_list_train_validation)]
# Test data
data_test = df[df['ID_CURSO_ALUNO'].isin(alunos_list_test)]


In [7]:
# Preprocess
# Train data
time_series_dataframe, np_estatico, labels = models.pipelineLibrary.pipe_default.fit_transform(data_train)
np_series_lstm = models.pipelineLibrary.pipe_lstm.fit_transform(time_series_dataframe)

np_dinamico_linear = np_series_lstm.reshape(np_series_lstm.shape[0],-1)
np_data = np.concatenate((np_estatico,np_dinamico_linear),axis=1)

# Save np_series_lstm, np_estatico, labels
np.savez('orig_train_syn_data.npz', np_series_lstm=np_series_lstm, np_estatico=np_estatico, label=labels)

# Test data
time_series_dataframe_test, np_estatico_test, label_test = models.pipelineLibrary.pipe_default.transform(data_test)
np_series_lstm_test = models.pipelineLibrary.pipe_lstm.transform(time_series_dataframe_test)
# Save np_series_lstm_test, np_estatico_test, label_test
np.savez('test_syn_data.npz', np_series_lstm=np_series_lstm_test, np_estatico=np_estatico_test, label=label_test)

In [8]:
print(np_series_lstm.shape)

print(np_dinamico_linear.shape)
print(np_estatico.shape)


(118, 4, 23)
(118, 92)
(118, 14)


In [50]:
new_labels = labels.copy()
enc = OneHotEncoder(dtype=int)
enc.fit(new_labels)
new_labels = enc.transform(new_labels).toarray()

In [51]:
r_np_estatico = np.repeat(np_estatico, repeats=window, axis=0)
r3d_np_estatico = r_np_estatico.reshape(-1, window, np_estatico.shape[1])


In [52]:
# Concatenate static features data in each step tail
np_series = np.concatenate((np_series_lstm, r3d_np_estatico), axis=2)
np_series.shape

(118, 4, 37)

In [53]:
[ensem_dat0, ensem_dat1, ensem_dat2,
 ensem_dat3, ensem_dat4, ensem_dat5,
 ensem_dat6, ensem_dat7, ensem_dat8, ensem_dat9] = np.array_split(np_series_lstm, 10)

[ensem_lab0, ensem_lab1, ensem_lab2,
 ensem_lab3, ensem_lab4, ensem_lab5,
 ensem_lab6, ensem_lab7, ensem_lab8, ensem_lab9] = np.array_split(new_labels, 10)

In [28]:
pathlib.Path('ensem').mkdir(parents=True, exist_ok=True) 

np.save('ensem/ensem_dat0.npy', ensem_dat0)
np.save('ensem/ensem_lab0.npy', ensem_lab0)

np.save('ensem/ensem_dat1.npy', ensem_dat1)
np.save('ensem/ensem_lab1.npy', ensem_lab1)

np.save('ensem/ensem_dat2.npy', ensem_dat2)
np.save('ensem/ensem_lab2.npy', ensem_lab2)

np.save('ensem/ensem_dat3.npy', ensem_dat3)
np.save('ensem/ensem_lab3.npy', ensem_lab3)

np.save('ensem/ensem_dat4.npy', ensem_dat4)
np.save('ensem/ensem_lab4.npy', ensem_lab4)

np.save('ensem/ensem_dat5.npy', ensem_dat5)
np.save('ensem/ensem_lab5.npy', ensem_lab5)

np.save('ensem/ensem_dat6.npy', ensem_dat6)
np.save('ensem/ensem_lab6.npy', ensem_lab6)

np.save('ensem/ensem_dat7.npy', ensem_dat7)
np.save('ensem/ensem_lab7.npy', ensem_lab7)

np.save('ensem/ensem_dat8.npy', ensem_dat8)
np.save('ensem/ensem_lab8.npy', ensem_lab8)

np.save('ensem/ensem_dat9.npy', ensem_dat9)
np.save('ensem/ensem_lab9.npy', ensem_lab9)

In [29]:
np.save('ensem/np_series_lstm.npy', np_series_lstm)

In [30]:
%run imbalanced-sequence-classification-master/run_autoenc.py ensem/np_series_lstm.npy teste1 Power

Exception: File `'imbalanced-sequence-classification-master/run_autoenc.py'` not found.

In [ ]:
%run imbalanced-sequence-classification-master/get_autoenc_adasyn_synthetic.py ensem/ teste1_autoenc-out/ teste1 Power

In [ ]:
synth_dat0 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat0.npy')
synth_lab0 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab0.npy')

synth_dat1 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat1.npy')
synth_lab1 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab1.npy')

synth_dat2 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat2.npy')
synth_lab2 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab2.npy')

synth_dat3 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat3.npy')
synth_lab3 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab3.npy')

synth_dat4 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat4.npy')
synth_lab4 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab4.npy')

synth_dat5 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat5.npy')
synth_lab5 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab5.npy')

synth_dat6 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat6.npy')
synth_lab6 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab6.npy')

synth_dat7 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat7.npy')
synth_lab7 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab7.npy')

synth_dat8 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_dat8.npy')
synth_lab8 = np.load('teste1_autoenc_syn_adasyn_ensem/ensem_lab8.npy')

In [ ]:
np_labels = np.delete(new_labels, 0, 1)
np_labels

In [ ]:
synth_lab0 = np.delete(synth_lab0, 0, 1)

synth_lab1 = np.delete(synth_lab1, 0, 1)

synth_lab2 = np.delete(synth_lab2, 0, 1)

synth_lab3 = np.delete(synth_lab3, 0, 1)

synth_lab4 = np.delete(synth_lab4, 0, 1)

synth_lab5 = np.delete(synth_lab5, 0, 1)

synth_lab6 = np.delete(synth_lab6, 0, 1)

synth_lab7 = np.delete(synth_lab7, 0, 1)

synth_lab8 = np.delete(synth_lab8, 0, 1)

In [ ]:
np_synth = np.concatenate((np_series_lstm, synth_dat0, synth_dat1,
                           synth_dat2, synth_dat3, synth_dat4,
                           synth_dat5, synth_dat6, synth_dat7, synth_dat8), axis=0)
np_synth_labels = np.concatenate((np_labels, synth_lab0, synth_lab1,
                                  synth_lab2, synth_lab3, synth_lab4,
                                  synth_lab5, synth_lab6, synth_lab7, synth_lab8), axis=0)


In [ ]:
np_synth_linear = np_synth.reshape(np_synth.shape[0], -1)

In [ ]:
np.save('np_synth.npy', np_synth)
np.save('np_synth_linear.npy', np_synth_linear)
np.save('np_synth_labels.npy', np_synth_labels)

# Save synthetic balanced train data
np.savez('train_syn_data.npz', np_series_lstm=np_synth, np_estatico=np_synth_linear, label=np_synth_labels)



In [ ]:
where_are_NaNs = np.isnan(np_synth_linear)
np_synth_linear[where_are_NaNs] = 0
x = np_synth_linear

In [ ]:
# Generate t-SNE visualization
from sklearn.manifold import TSNE

tsne_evasao = TSNE(n_components=2, n_iter=1000, perplexity=100, verbose=2).fit_transform(x)

In [ ]:
tsne_evasao_df = pd.DataFrame(data = tsne_evasao,
                              columns = ['x', 'y'])

tsne_evasao_df.tail()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.figure(figsize=(10,10))
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.xlabel('component 1',fontsize=20)
plt.ylabel('component 2',fontsize=20)
plt.title("Preprocessed data t-SNE",fontsize=20)
targets_idx = [0, 1]
targets_name = ["Success", "Dropout"]
colors = ['green', 'blue']
for target, color in zip(targets_idx, colors):
    indicesToKeep = np_synth_labels == target
    plt.scatter(tsne_evasao_df.loc[indicesToKeep, 'x'],
                tsne_evasao_df.loc[indicesToKeep, 'y'], edgecolors=color, s = 10, alpha=0.5, facecolors='none')

plt.legend(targets_name,prop={'size': 15})